In [1]:
import pandas as pd
import numpy as np

In [2]:
from typing import Union

import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from sklearn.preprocessing import LabelEncoder

from modelling.models import TextProductMatch

MAX_LEN = 70
MODEL_NAME = 'bert-base-uncased'
N_CLASSES = 1000
EPOCHS = 1
BATCH_SIZE = 32

params = {
    "MAX_LEN": MAX_LEN,
    "MODEL_NAME": MODEL_NAME,
    "N_CLASSES": N_CLASSES,
    "POOLING": "gem"
}

word_model = transformers.TFAutoModel.from_pretrained(MODEL_NAME)
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)


def encoder(titles: Union[str]):
    ct = len(titles)

    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    att_masks = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32')

    for i in range(len(titles)):
        enc = tokenizer.encode_plus(titles[i],
                                    padding="max_length",
                                    max_length=MAX_LEN,
                                    truncation=True,
                                    add_special_tokens=True,
                                    return_tensors='tf',
                                    return_attention_mask=True,
                                    return_token_type_ids=True)
        input_ids[i] = enc["input_ids"]
        att_masks[i] = enc["attention_mask"]
        token_type_ids[i] = enc["token_type_ids"]

    return input_ids, att_masks, token_type_ids

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [8]:
ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

labels_onehot = tf.keras.layers.Input(shape=(N_CLASSES), dtype=tf.int32)
# labels_onehot = tf.one_hot(labels, depth=N_CLASSES)

x = word_model(ids, attention_mask=att, token_type_ids=tok)[0]

In [9]:
x = tf.keras.layers.GlobalAveragePooling1D()(x)

TensorShape([None, 768])